In [1]:
# 這個Jupyter Notebook的環境
import platform
import tensorflow
import keras
print("Platform: {}".format(platform.platform()))
print("Tensorflow version: {}".format(tensorflow.__version__))
print("Keras version: {}".format(keras.__version__))

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from IPython.display import Image

/home/dzw/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Platform: Linux-4.13.0-38-generic-x86_64-with-debian-stretch-sid
Tensorflow version: 1.7.0
Keras version: 2.1.5


Using TensorFlow backend.


In [2]:
import os
import shutil

data_path = os.path.join('../data/train')

In [3]:
#训练所需数据的地址
def mkdir_if_not_exist(path):
    if not os.path.exists(os.path.join(*path)):
        os.makedirs(os.path.join(*path))
train_dir = ['data','train']
test_dir = ['data', 'test']
valid_dir = ['data', 'valid']

mkdir_if_not_exist(train_dir)
mkdir_if_not_exist(test_dir)
mkdir_if_not_exist(valid_dir)

In [5]:
#在每个文件夹下创建dog cat 文件夹分类
for dir in os.listdir('data'):
    dog_dir = ['data',dir,'dog']
    cat_dir = ['data',dir,'cat']
    mkdir_if_not_exist(dog_dir)
    mkdir_if_not_exist(cat_dir)

In [11]:
# 实际未分类数据所在地址
all_data = '../data/train'

In [12]:
import os
import random
import shutil

def organize_folder(folder):
    _, _, filenames = next(os.walk(folder))
    unique_classes = {filename.split(".")[0] for filename in filenames}
    for _class in unique_classes:
        path = os.path.join(folder, _class)
        if not os.path.exists(path):
            os.makedirs(path)
        for filename in filenames:
            if filename.startswith(_class):
                shutil.move(os.path.join(folder, filename), os.path.join(path, filename))        
    
def create_sample_folder(_from, to, percentage=0.1, move=True):
    if not os.path.exists(to):
        os.makedirs(to)
    _, folders, _ = next(os.walk(_from))
    for folder in folders:
        if not os.path.exists(os.path.join(to, folder)):
            os.makedirs(os.path.join(to, folder))
        _, _, files = next(os.walk(os.path.join(_from, folder)))
        sample = random.sample(files, int(len(files) * percentage))
        for filename in sample:
            if move:
                shutil.move(os.path.join(_from, folder, filename), os.path.join(to, folder, filename))
            else:
                shutil.copyfile(os.path.join(_from, folder, filename), os.path.join(to, folder, filename))

In [13]:
organize_folder(all_data)

In [16]:
create_sample_folder(all_data, os.path.join(*train_dir))
create_sample_folder(all_data, os.path.join(*test_dir))
create_sample_folder(all_data, os.path.join(*valid_dir))

In [23]:
ls 'data' | head

test/
train/
valid/


In [24]:
ls 'data/train' | head

cat/
dog/


In [26]:
from keras.preprocessing.image import ImageDataGenerator

#归一化处理
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

#keras 数据读取机制
train_gen = train_datagen.flow_from_directory(
    os.path.join(*train_dir),
    target_size=(150,150),
    batch_size=24,
    class_mode='binary'
)

valid_gen = test_datagen.flow_from_directory(
    os.path.join(*valid_dir),
    target_size=(150, 150),
    batch_size=24,
    class_mode='binary'
)

Found 2500 images belonging to 2 classes.
Found 2024 images belonging to 2 classes.


In [27]:
# 查看一次的输出
for data, label in train_gen:
    print('data batch shape:', data.shape)
    print('labels batch shape:', label.shape)
    break

data batch shape: (24, 150, 150, 3)
labels batch shape: (24,)
